# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from time import sleep

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# index_col=False
#index_col=None
city_data = pd.read_csv("../output_data_files/CityWeatherData.csv", usecols=range(1,10))
city_data.head()


,City,Lat,Lng,Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Norman Wells,65.2820,-126.8329,32.04,86,75,10.36,CA,2021-09-26 13:22:03
1,Thompson,55.7435,-97.8558,53.76,71,40,5.75,CA,2021-09-26 13:18:39
2,Hirara,24.8000,125.2833,83.44,77,3,11.32,JP,2021-09-26 13:22:04
3,George Town,5.4112,100.3354,76.95,98,40,5.75,MY,2021-09-26 13:28:23
4,Cape Town,-33.9258,18.4232,64.27,72,0,1.99,ZA,2021-09-26 13:28:01


In [3]:
city_data.dtypes

City                 object
Lat                 float64
Lng                 float64
Max Temp (°F)       float64
Humidity (%)          int64
Cloudiness (%)        int64
Wind Speed (mph)    float64
Country              object
Date                 object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configure to use api
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = city_data[["Lat","Lng"]]


#Add Heatmap layer to map
humidity = city_data["Humidity (%)"].astype(float)

In [5]:
# configure map (map will repeat)
fig = gmaps.figure(center = [0,0] ,zoom_level = 2, mouse_handling='GREEDY')

#Add heat layer over map , set max intensity to dataset max
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)

fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
city_data.head()

,City,Lat,Lng,Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Norman Wells,65.2820,-126.8329,32.04,86,75,10.36,CA,2021-09-26 13:22:03
1,Thompson,55.7435,-97.8558,53.76,71,40,5.75,CA,2021-09-26 13:18:39
2,Hirara,24.8000,125.2833,83.44,77,3,11.32,JP,2021-09-26 13:22:04
3,George Town,5.4112,100.3354,76.95,98,40,5.75,MY,2021-09-26 13:28:23
4,Cape Town,-33.9258,18.4232,64.27,72,0,1.99,ZA,2021-09-26 13:28:01


In [7]:
city_data.describe()

,Lat,Lng,Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph)
count,575.000000,575.000000,575.000000,575.000000,575.000000,575.000000
mean,19.692363,22.797924,66.178557,68.970435,53.634783,7.981113
std,33.042262,90.051911,18.439235,23.700016,37.134967,5.658810
min,-54.800000,-175.200000,20.050000,1.000000,0.000000,0.000000
25%,-9.060700,-53.205800,53.660000,55.000000,18.000000,3.990000
50%,22.075200,27.849200,69.530000,76.000000,60.000000,6.710000
75%,47.753550,100.427150,79.130000,87.000000,90.000000,10.805000
max,77.484000,179.316700,107.760000,100.000000,100.000000,39.500000


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# configuration of perfect conditions (my perfect conditions)
#A max temperature lower than 80 degrees but higher than 70.  EQUAL TO AS WELL????
#Wind speed less than 10 mph.
#Zero cloudiness.

ideal_weather_city = city_data[(city_data['Max Temp (°F)'] > 70) & (city_data['Max Temp (°F)'] < 80) & \
                               (city_data['Cloudiness (%)'] == 0) & (city_data['Wind Speed (mph)'] < 10)]

#drop any rows that may have null values
ideal_weather_city.dropna()

ideal_weather_city.reset_index(drop=True, inplace=True)

ideal_weather_city


,City,Lat,Lng,Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Kalabo,-14.9700,22.6814,79.27,11,0,2.73,ZM,2021-09-26 13:23:23
1,Koungou,-12.7336,45.2042,74.91,73,0,4.61,YT,2021-09-26 13:23:38
2,Beloha,-25.1667,45.0500,73.42,66,0,9.82,MG,2021-09-26 13:24:49
3,Panjakent,39.4952,67.6093,73.20,16,0,5.01,TJ,2021-09-26 13:32:57
4,Al Bardīyah,31.7561,25.0865,74.23,56,0,8.30,LY,2021-09-26 13:38:03
5,Mount Isa,-20.7333,139.5000,73.17,15,0,9.22,AU,2021-09-26 13:38:17
6,Betioky,-23.7167,44.3833,73.36,30,0,7.85,MG,2021-09-26 13:39:23
7,Waddān,29.1614,16.1390,79.12,26,0,8.34,LY,2021-09-26 13:40:48


In [9]:
ideal_weather_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   City              8 non-null      object 
 1   Lat               8 non-null      float64
 2   Lng               8 non-null      float64
 3   Max Temp (°F)     8 non-null      float64
 4   Humidity (%)      8 non-null      int64  
 5   Cloudiness (%)    8 non-null      int64  
 6   Wind Speed (mph)  8 non-null      float64
 7   Country           8 non-null      object 
 8   Date              8 non-null      object 
dtypes: float64(4), int64(2), object(3)
memory usage: 704.0+ bytes


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.
#Store into variable named hotel_df
hotel_df = ideal_weather_city.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Kalabo,ZM,-14.9700,22.6814,
1,Koungou,YT,-12.7336,45.2042,
2,Beloha,MG,-25.1667,45.0500,
3,Panjakent,TJ,39.4952,67.6093,
4,Al Bardīyah,LY,31.7561,25.0865,
5,Mount Isa,AU,-20.7333,139.5000,
6,Betioky,MG,-23.7167,44.3833,
7,Waddān,LY,29.1614,16.1390,


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# 5000 metres away from coordinates: is this in metres or miles????
# what is default, metric or imperial
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [12]:
# Want to only 10 pins on the heatmap, save calls to api ($)

cnt = 1
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    if cnt <=10:
        params["location"] = f"{lat},{lng}"
        print(f"Getting Results for Index {index}: {city_name}.")
        try:
            response = requests.get(base_url, params=params)
            if response.status_code == 200:
                result_j = response.json()
                results = result_j['results']
                if results:
                    #Save the hotel name to the dataframe
                    cnt += 1 
                    print(f"Closest hotel in {city_name} is {results[0]['name']}.")
                    print("")
                    hotel_df.loc[index, "Hotel Name"] = results[0]['name']
                else:
                    print("Could not find result")
                    print("")
            else:
                print("Could not find result")
                print("")
        except (KeyError, IndexError):
            print("Could not find result")
            print("")
            #delay the code to avoid an error
            time.sleep(2)
        
print("")
print("Search Complete")

Getting Results for Index 0: Kalabo.
Closest hotel in Kalabo is Sikakubete Guest House Kalabo Town Council.

Getting Results for Index 1: Koungou.
Closest hotel in Koungou is Hôtel Caribou.

Getting Results for Index 2: Beloha.
Could not find result

Getting Results for Index 3: Panjakent.
Closest hotel in Panjakent is Umarion.

Getting Results for Index 4: Al Bardīyah.
Closest hotel in Al Bardīyah is Hotel Burdy.

Getting Results for Index 5: Mount Isa.
Closest hotel in Mount Isa is Isa Hotel.

Getting Results for Index 6: Betioky.
Could not find result

Getting Results for Index 7: Waddān.
Closest hotel in Waddān is بشير التاجوري.


Search Complete


In [13]:
# have 10 hotel results to pin (may not be in order in DF):
hotel_df.head(11)

,City,Country,Lat,Lng,Hotel Name
0,Kalabo,ZM,-14.9700,22.6814,Sikakubete Guest House Kalabo Town Council
1,Koungou,YT,-12.7336,45.2042,Hôtel Caribou
2,Beloha,MG,-25.1667,45.0500,
3,Panjakent,TJ,39.4952,67.6093,Umarion
4,Al Bardīyah,LY,31.7561,25.0865,Hotel Burdy
5,Mount Isa,AU,-20.7333,139.5000,Isa Hotel
6,Betioky,MG,-23.7167,44.3833,
7,Waddān,LY,29.1614,16.1390,بشير التاجوري


In [14]:
#drop rows that do not have a hotel name
#df.dropna(subset = ["column2"], inplace=True)
#hotel_df.dropna(subset=['Hotel Name'], inplace= True)
#hotel_df

# Get indexes where name column has value john
#indexNames = df[df['name'] == 'john'].index
index_blank = hotel_df[hotel_df['Hotel Name'] == ""].index
# Delete these row indexes from dataFrame
#df.drop(indexNames , inplace=True)
hotel_df.drop(index_blank , inplace=True)

In [15]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Kalabo,ZM,-14.9700,22.6814,Sikakubete Guest House Kalabo Town Council
1,Koungou,YT,-12.7336,45.2042,Hôtel Caribou
3,Panjakent,TJ,39.4952,67.6093,Umarion
4,Al Bardīyah,LY,31.7561,25.0865,Hotel Burdy
5,Mount Isa,AU,-20.7333,139.5000,Isa Hotel
7,Waddān,LY,29.1614,16.1390,بشير التاجوري


In [16]:
len(hotel_df)

6

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))